In [1]:
import pandas as pd

In [2]:
import openpyxl

In [73]:
import json
from pandas.io.json import json_normalize

In [27]:
import warnings
warnings.filterwarnings("ignore")
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [3]:
('Downloads/datos_municipales_mx-master/datos_municipales_mx-master/datos/siha_2_2_4_2.xlsx')

'Downloads/datos_municipales_mx-master/datos_municipales_mx-master/datos/siha_2_2_4_2.xlsx'

In [4]:
from openpyxl import Workbook

workbook = Workbook()
sheet = workbook.active

sheet["A1"] = "hello"
sheet["B1"] = "world!"

workbook.save(filename="hello_world.xlsx")

'C:\\Users\\KNEAR\\Downloads\\datos_municipales_mx-master\\datos_municipales_mx-master\\datos'

In [8]:
from openpyxl import load_workbook

In [11]:
workbook = load_workbook(filename="reviews-sample.xlsx")


In [92]:
workbook.sheetnames

['amazon_reviews_us_Watches_v (2)', 'amazon']

In [13]:
sheet = workbook.active

In [16]:
sheet.title

'amazon'

In [28]:
sheet2 = workbook.get_sheet_by_name('amazon_reviews_us_Watches_v (2)')

In [29]:
sheet2.title

'amazon_reviews_us_Watches_v (2)'

In [30]:
sheet.cell(row=10, column=6)

<Cell 'amazon'.F10>

In [31]:
sheet.cell(row=10, column=6).value

"G-Shock Men's Grey Sport Watch"

In [32]:
sheet["F10"].value

"G-Shock Men's Grey Sport Watch"

In [34]:
for row in sheet.iter_rows(min_row=1,
                            max_row=2,
                            min_col=1,
                            max_col=3):
    print(row)

(<Cell 'amazon'.A1>, <Cell 'amazon'.B1>, <Cell 'amazon'.C1>)
(<Cell 'amazon'.A2>, <Cell 'amazon'.B2>, <Cell 'amazon'.C2>)


In [35]:
for col in sheet.iter_cols(min_row=1,
                            max_row=2,
                            min_col=1,
                            max_col=3):
    print(col)

(<Cell 'amazon'.A1>, <Cell 'amazon'.A2>)
(<Cell 'amazon'.B1>, <Cell 'amazon'.B2>)
(<Cell 'amazon'.C1>, <Cell 'amazon'.C2>)


In [45]:
for value in sheet.iter_rows(min_row=1,
                            max_row=2,
                            min_col=1,
                            max_col=3,
                           values_only=True):
    print(value)

('marketplace', 'customer_id', 'review_id')
('US', 3653882, 'R3O9SGZBVQBV76')


In [39]:
for value in sheet.iter_cols(min_row=1,
                            max_row=2,
                            min_col=1,
                            max_col=3,
                          values_only=True):
    print(value)

('marketplace', 'US')
('customer_id', 3653882)
('review_id', 'R3O9SGZBVQBV76')


In [72]:
# Using the values_only because you want to return the cells' values
for row in sheet.iter_rows(min_row=2,
                           min_col=4,
                           max_col=7,
                           values_only=True):
#     print(row[0])
    None

In [55]:
products = {}

# Using the values_only because you want to return the cells' values
for row in sheet.iter_rows(min_row=2,
                           max_row=3,
                           min_col=4,
                           max_col=7,
                           values_only=True):
    product_id = row[0]
    product = {
        "parent": row[1],
        "title": row[2],
        "category": row[3]
    }
    products[product_id] = product

# Using json here to be able to format the output for displaying later
print(json.dumps(products))

{"B00FALQ1ZC": {"parent": 937001370, "title": "Invicta Women's 15150 \"Angel\" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch", "category": "Watches"}, "B00D3RGO20": {"parent": 484010722, "title": "Kenneth Cole New York Women's KC4944 Automatic Silver Automatic Mesh Bracelet Analog Watch", "category": "Watches"}}


In [56]:
products

{'B00FALQ1ZC': {'parent': 937001370,
  'title': 'Invicta Women\'s 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch',
  'category': 'Watches'},
 'B00D3RGO20': {'parent': 484010722,
  'title': "Kenneth Cole New York Women's KC4944 Automatic Silver Automatic Mesh Bracelet Analog Watch",
  'category': 'Watches'}}

In [68]:
df = pd.DataFrame.from_dict(products, orient='columns')

In [71]:
df

,B00FALQ1ZC,B00D3RGO20
parent,937001370,484010722
title,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Kenneth Cole New York Women's KC4944 Automatic...
category,Watches,Watches


In [70]:
df.T

,parent,title,category
B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches
B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches


In [78]:
import datetime
from dataclasses import dataclass

@dataclass
class Product:
    id: str
    parent: str
    title: str
    category: str

@dataclass
class Review:
    id: str
    customer_id: str
    stars: int
    headline: str
    body: str
    date: datetime.datetime

In [80]:
# Product fields
PRODUCT_ID = 3
PRODUCT_PARENT = 4
PRODUCT_TITLE = 5
PRODUCT_CATEGORY = 6

# Review fields
REVIEW_ID = 2
REVIEW_CUSTOMER = 1
REVIEW_STARS = 7
REVIEW_HEADLINE = 12
REVIEW_BODY = 13
REVIEW_DATE = 14

In [85]:
products = []
reviews = []

# Using the values_only because you just want to return the cell value
for row in sheet.iter_rows(min_row=2, values_only=True):
    product = Product(id=row[PRODUCT_ID],
                      parent=row[PRODUCT_PARENT],
                      title=row[PRODUCT_TITLE],
                      category=row[PRODUCT_CATEGORY])
    products.append(product)

    # You need to parse the date from the spreadsheet into a datetime format
    spread_date = row[REVIEW_DATE]
    parsed_date = datetime.datetime.strptime(spread_date, "%Y-%m-%d")

    review = Review(id=row[REVIEW_ID],
                    customer_id=row[REVIEW_CUSTOMER],
                    stars=row[REVIEW_STARS],
                    headline=row[REVIEW_HEADLINE],
                    body=row[REVIEW_BODY],
                    date=parsed_date)
    reviews.append(review)

print(products[0])
print(reviews[0])

Product(id='B00FALQ1ZC', parent=937001370, title='Invicta Women\'s 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch', category='Watches')
Review(id='R3O9SGZBVQBV76', customer_id=3653882, stars=5, headline='Five Stars', body='Absolutely love this watch! Get compliments almost every time I wear it. Dainty.', date=datetime.datetime(2015, 8, 31, 0, 0))


In [86]:
'https://realpython.com/openpyxl-excel-spreadsheets-python/'

'https://realpython.com/openpyxl-excel-spreadsheets-python/'

In [89]:
wb = load_workbook(filename='hello_world.xlsx')

In [137]:
sheet.freeze_panes = 'A1'

In [128]:
workbook.save(filename="reviews-sample.xlsx")

In [131]:
sheet.dimensions

'A1:O100'

In [132]:
sheet.auto_filter.ref = "A1:O100"
workbook.save(filename="reviews-sample.xlsx")

In [134]:
from openpyxl.utils import FORMULAE
len(FORMULAE)

352

In [139]:
sheet["P2"] = "=AVERAGE(H2:H100)"
sheet["P3"] = '=COUNTIF(I2:I100, ">0")'
workbook.save(filename="reviews-sample.xlsx")

In [141]:
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors

In [142]:
# Create a few styles
bold_font = Font(bold=True)
big_red_text = Font(color=colors.RED, size=20)
center_aligned_text = Alignment(horizontal="center", vertical='center')
double_border_side = Side(border_style="double")
square_border = Border(top=double_border_side,
                        right=double_border_side,
                        bottom=double_border_side,
                        left=double_border_side)

# Style some cells!
sheet["A2"].font = bold_font
sheet["A3"].font = big_red_text
sheet["A4"].alignment = center_aligned_text
sheet["A5"].border = square_border
workbook.save(filename="sample_styles.xlsx")

In [143]:
from openpyxl.styles import NamedStyle

In [145]:
header = NamedStyle(name="header")
header.font = Font(bold=True)
header.border = Border(bottom=Side(border_style="thin"))
header.alignment = Alignment(horizontal="center", vertical="center")

# Now let's apply this to all first row (header) cells
header_row = sheet[1]
for cell in header_row:
     cell.style = header

workbook.save(filename="sample_styles.xlsx")

In [148]:
from openpyxl.styles import PatternFill, colors
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule

red_background = PatternFill(bgColor=colors.BLUE)
diff_style = DifferentialStyle(fill=red_background)
rule = Rule(type="expression", dxf=diff_style)
rule.formula = ["$H1=5"]
sheet.conditional_formatting.add("A1:O100", rule)
workbook.save("sample_styles.xlsx")

In [149]:
from openpyxl.formatting.rule import ColorScaleRule
color_scale_rule = ColorScaleRule(start_type="min",
                                   start_color=colors.RED,
                                   end_type="max",
                                   end_color=colors.GREEN)

# Again, let's add this gradient to the star ratings, column "H"
sheet.conditional_formatting.add("H2:H100", color_scale_rule)
workbook.save("sample_styles.xlsx")

In [161]:
!pip install Pillow

In [162]:
from openpyxl.drawing.image import Image


In [152]:
# Let's use the hello_world spreadsheet since it has less data
wb = load_workbook(filename="hello_world.xlsx")
sheet_wb = wb.active

In [171]:
logo = openpyxl.drawing.image.Image('lg.jpg')

In [173]:
logo.height = 150
logo.width = 150

sheet_wb.add_image(logo, "A3")
wb.save(filename="hello_world_logo.xlsx")